code for only one day, needs input of RD and IFF files, directory location of files, orig and dest airport appreivation, and date

exports only one file 

In [ ]:
import pandas
import gzip
import shutil
import os

directory_files   = "/raid/mo0dy/tryouts/"
iffFileName = 'IFF_USA_20250201_050005_86393_LADDfiltered.csv'
rdFileName = 'RD_USA_20250201_050005_86393_LADDfiltered.csv' 
outputFileName = "/raid/mo0dy/cleaned/20250201_050005_86393.csv"

iffFileName_dir = f'{directory_files}{iffFileName}'
rdFileName_dir = f'{directory_files}{rdFileName}'

def decompress_and_replace_file(file_path):
    if file_path.endswith('.csv.gz'):
        decompressed_file_path = file_path.replace('.gz', '')

        with gzip.open(file_path, 'rb') as f_in:
            with open(decompressed_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        os.remove(file_path)
        return decompressed_file_path
    else:
        return file_path

def readRdFile(rdFileName_dir):
    result = pandas.read_csv(rdFileName_dir, usecols=["Msn", "Orig", "EstOrig", "Dest", "EstDest"])
    msnVals = [] 
    for i in range(result['Msn'].size):
        msnVals.append(result['Msn'][i])
    return msnVals

def getMergedCsvFile(flightKeys):
    dictToTurnIntoDataFrame = {'fltKey': [], 'Time': [], 'Latitude': [],
                               'Longitude': [], 'Altitude': [], 'RecTypeCat': [],
                               'Significance': [], 'GroundSpeed': [], 'FlightCourse': []}
    iffFile = open(iffFileName_dir)

    while 1:
        print("\nrunning...\n")
        lines = iffFile.readlines(100000)
        if not lines:
            break
        for line in lines:
            iffWrite = line.split(",")
            if iffWrite[0] == "3" and int(iffWrite[2]) in flightKeys:
                dictToTurnIntoDataFrame['fltKey'].append(iffWrite[2]) 
                dictToTurnIntoDataFrame['Time'].append(int(float(iffWrite[1])))
                dictToTurnIntoDataFrame['Latitude'].append(float(iffWrite[9]))
                dictToTurnIntoDataFrame['Longitude'].append(float(iffWrite[10]))
                dictToTurnIntoDataFrame['Altitude'].append(float(iffWrite[11]))
                dictToTurnIntoDataFrame['RecTypeCat'].append(iffWrite[8])
                dictToTurnIntoDataFrame['Significance'].append(iffWrite[12])
                dictToTurnIntoDataFrame['GroundSpeed'].append(iffWrite[16])
                dictToTurnIntoDataFrame['FlightCourse'].append(iffWrite[17]) 
    iffFile.close()

    df = pandas.DataFrame(dictToTurnIntoDataFrame, columns=['fltKey', 'Time', 'Latitude', 
                                                            'Longitude','Altitude', 'RecTypeCat', 
                                                            'Significance','GroundSpeed', 'FlightCourse'])
    df.to_csv(outputFileName, index=None, header=True) 
    return df.head(15)
    print('{} traces exported'.format(len(set(df['fltKey']))))

msnVals = readRdFile(rdFileName_dir)
# print(msnVals)
getMergedCsvFile(msnVals)

code for multiple days, needs input of RD and IFF files, directory location of files, orig and dest airport appreivation, and dates

exports only one file 

In [ ]:
import os
import pandas as pd
import gzip
import shutil

directory_files = '/Users/annaschneider/Desktop/relavent_files_raftparser'
iffFileNames = [
    '/IFF1/IFF_USA_20230201_050001_86397_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230202_050003_86396_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230203_050002_86396_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230204_050000_86398_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230205_050000_86398_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230207_050001_86398_LADDfiltered.csv',
    '/IFF1/IFF_USA_20230208_050001_86398_LADDfiltered.csv'
]
rdFileNames = [
    '/RD/RD_USA_20230201_050001_86397_LADDfiltered.csv',
    '/RD/RD_USA_20230202_050003_86396_LADDfiltered.csv',
    '/RD/RD_USA_20230203_050002_86396_LADDfiltered.csv',
    '/RD/RD_USA_20230204_050000_86398_LADDfiltered.csv',
    '/RD/RD_USA_20230205_050000_86398_LADDfiltered.csv',
    '/RD/RD_USA_20230207_050001_86398_LADDfiltered.csv',
    '/RD/RD_USA_20230208_050001_86398_LADDfiltered.csv'
    
]

orig = 'SFO'
dest = 'SYR'
start_date = '20230201'
end_date = '20230208'

outputFileName = f'{directory_files}/{orig}_{dest}_{start_date}_{end_date}.csv'
iffFileNames_dir = [f'{directory_files}{iffFileName}' for iffFileName in iffFileNames]
rdFileNames_dir = [f'{directory_files}{rdFileName}' for rdFileName in rdFileNames]

def decompress_and_replace_file(file_path):
    if file_path.endswith('.csv.gz'):
        decompressed_file_path = file_path.replace('.gz', '')

        with gzip.open(file_path, 'rb') as f_in:
            with open(decompressed_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        os.remove(file_path)
        return decompressed_file_path
    else:
        return file_path

def readRd_merge_df(rdFileName_dir, iffFileName_dir):
    result = pd.read_csv(rdFileName_dir, usecols=["Msn", "Orig", "EstOrig", "Dest", "EstDest"])
    msnVals = []
    for i in range(result['Msn'].size):
        if (result["EstOrig"][i] == orig or result["Orig"][i] == orig) and \
                (result["EstDest"][i] == dest or result["Dest"][i] == dest):
            msnVals.append(result['Msn'][i])

    dictToTurnIntoDataFrame = {'fltKey': [], 'Time': [], 'Latitude': [],
                               'Longitude': [], 'Altitude': [], 'RecTypeCat': [],
                               'Significance': [], 'GroundSpeed': [], 'FlightCourse': []}
    
    # iffFile = gzip.open(iffFileName_dir, mode='rt')
    
    iffFile = open(iffFileName_dir)

    while 1:
        lines = iffFile.readlines(50000)
        if not lines:
            break
        for line in lines:
            iffWrite = line.split(",")
            if iffWrite[0] == "3" and int(iffWrite[2]) in msnVals:
                dictToTurnIntoDataFrame['fltKey'].append(iffWrite[2]) 
                dictToTurnIntoDataFrame['Time'].append(int(float(iffWrite[1])))
                dictToTurnIntoDataFrame['Latitude'].append(float(iffWrite[9]))
                dictToTurnIntoDataFrame['Longitude'].append(float(iffWrite[10]))
                dictToTurnIntoDataFrame['Altitude'].append(float(iffWrite[11]))
                dictToTurnIntoDataFrame['RecTypeCat'].append(iffWrite[8])
                dictToTurnIntoDataFrame['Significance'].append(iffWrite[12])
                dictToTurnIntoDataFrame['GroundSpeed'].append(iffWrite[16])
                dictToTurnIntoDataFrame['FlightCourse'].append(iffWrite[17]) 
    iffFile.close()

    df = pandas.DataFrame(dictToTurnIntoDataFrame, columns=['fltKey', 'Time', 'Latitude', 
                                                            'Longitude','Altitude', 'RecTypeCat', 
                                                            'Significance','GroundSpeed', 'FlightCourse']) 
    return df

iffFileNames_dir = [decompress_and_replace_file(iffFileName) for iffFileName in iffFileNames_dir]
rdFileNames_dir = [decompress_and_replace_file(rdFileName) for rdFileName in rdFileNames_dir]

all_dfs = []

for rdFileName_dir, iffFileName_dir in zip(rdFileNames_dir, iffFileNames_dir):
    df = readRd_merge_df(rdFileName_dir, iffFileName_dir)
    all_dfs.append(df)

main_df = pd.concat(all_dfs, ignore_index=True)
main_df.to_csv(outputFileName, index=None, header=True)

print(main_df.head(15))
print('{} traces exported'.format(len(set(main_df['fltKey']))))

    fltKey          Time  Latitude  Longitude  Altitude RecTypeCat  \
0   138732  1.675290e+09  37.61558 -122.38283      0.06          1   
1   138732  1.675290e+09  37.61556 -122.38278      0.06          1   
2   138732  1.675290e+09  37.61560 -122.38274      0.06          1   
3   138732  1.675290e+09  37.61560 -122.38273      0.06          1   
4   138732  1.675290e+09  37.61562 -122.38270      0.06          1   
5   138732  1.675290e+09  37.61563 -122.38269      0.06          1   
6   138732  1.675290e+09  37.61564 -122.38268      0.06          1   
7   138732  1.675290e+09  37.61563 -122.38264      0.06          1   
8   138732  1.675290e+09  37.61563 -122.38263      0.06          1   
9   138732  1.675290e+09  37.61563 -122.38262      0.06          1   
10  138732  1.675290e+09  37.61564 -122.38261      0.06          1   
11  138732  1.675290e+09  37.61564 -122.38260      0.06          1   
12  138732  1.675290e+09  37.61564 -122.38259      0.06          1   
13  138732  1.675290